# Desafio Sprint 3

## Etapa 1: ambiente

Nesta etapa devemos certificar que as bibliotecas **pandas** e **matplotlib** estão instaladas no ambiente atual.
Podemos importar ambas as bibliotecas com o comando:
```bash
python.exe -m pip install matplotlib pandas
```

Ao longo do projeto, também serão necessárias algumas bibliotecas adicionais.
```bash
python.exe -m pip install tabulate
```

Portanto recomenda-se instalar os pacotes necessários lendo o arquivo [requirements.txt](requirements.txt):
```bash
# Rodar comando na pasta do desafio
python.exe -m pip install -r requirements.txt
```

Podemos conferir a lista de pacotes instalados usando o comando:
```bash
python.exe -m pip freeze
```

## Etapa 2: desenvolvimento

Funcões utilitárias:

In [109]:
import pandas as pd # biblioteca de tratamento de dados
from IPython.display import display, Markdown # Exibição dos dados em tabelas markdown
from random import randint  # Funcao para gerar numeros aleatorios
import re as regex

#Nome das colunas do CSV como constantes
APP =                  "App"
CATEGORY =        "Category"
RATING =            "Rating"
REVIEWS =          "Reviews"
SIZE =               "Size"
INSTALLS =        "Installs"
TYPE =                "Type"
PRICE =              "Price"
AGE_RANGE = "Content Rating"
GENRE =            "Genres"
L_UPDATE =    "Last Updated"
C_VERSION =    "Current Ver"
A_VERSION =    "Android Ver"

def display_df_table(table: pd.DataFrame, show_index=True) -> None:
    display(Markdown(table.to_markdown(index=show_index)))

### 1. Leitura e Tratamento dos Dados de [googleplaystore.csv](./googleplaystore.csv)

Como primeiro passo devemos importar os dados. 

In [110]:
# Importando arquivo csv e convertendo para objeto dataframe
raw_dataframe = pd.read_csv("googleplaystore.csv") # Caminho relativo ao path deste notebook

# Mostrando primeiro registro do csv
display_df_table(raw_dataframe.head(1), False)

| App                                            | Category       |   Rating |   Reviews | Size   | Installs   | Type   |   Price | Content Rating   | Genres       | Last Updated    | Current Ver   | Android Ver   |
|:-----------------------------------------------|:---------------|---------:|----------:|:-------|:-----------|:-------|--------:|:-----------------|:-------------|:----------------|:--------------|:--------------|
| Photo Editor & Candy Camera & Grid & ScrapBook | ART_AND_DESIGN |      4.1 |       159 | 19M    | 10,000+    | Free   |       0 | Everyone         | Art & Design | January 7, 2018 | 1.0.0         | 4.0.3 and up  |

Em seguida podemos remover todos os registros duplicados

In [111]:
# Removendo duplicatas
distinct_dataframe = raw_dataframe.drop_duplicates()
removed_registers = (len(raw_dataframe) - len(distinct_dataframe))

print(f"Número de Registros Bruto: {len(raw_dataframe):>14}")
print(f"Número de Registros Distintos: {len(distinct_dataframe):>10}")
print(f"Número de Registros Removidos: {removed_registers:>10}")

Número de Registros Bruto:          10841
Número de Registros Distintos:      10358
Número de Registros Removidos:        483


Podemos agora analisar o formato dos dados de cada coluna e tratar/filtrar de forma mais profunda

#### 1. Coluna "App"

Vamos observar os dados dessa coluna:

In [112]:
# Exibindo 10 registros variados da coluna "App"
display_df_table(distinct_dataframe[::randint(700, 1000)][APP].head(10))
# Este bloco pode ser executado diversas vezes para ter maior noção dos dados

|      | App                                            |
|-----:|:-----------------------------------------------|
|    0 | Photo Editor & Candy Camera & Grid & ScrapBook |
|  958 | Netflix                                        |
| 1885 | Pou                                            |
| 2851 | Beauty Makeup – Photo Makeover                 |
| 3762 | BeyondPod Podcast Manager                      |
| 4625 | T. Rowe Price Personal® App                    |
| 5460 | AP Installer                                   |
| 6297 | Zalo – Video Call                              |
| 7141 | CB Land                                        |
| 7977 | CV Builder for Smart Resumes                   |

Podemos notar que nesta coluna praticamente tem um pouco de tudo. 
Como o nome do aplicativo é uma informação muito importante para sua identificação, vamos apenas checar se a coluna possue algum valor nulo.

In [113]:
# Conta quantos valores em "App" sao nulos
missing_count = distinct_dataframe[APP].isnull().sum()

print(f"Valores nulos na coluna {APP}: {missing_count}")


Valores nulos na coluna App: 0


O arquivo não contém nenhum valor nulo na coluna "App", portanto não precisamos realizar nenhum tratamento.

#### 2. Coluna "Category"

Vamos observar os dados desta coluna:

In [139]:
# Exibindo 10 registros variados da coluna "Category"
display_df_table(distinct_dataframe[::randint(700, 1000)][CATEGORY].head(10))
# Este bloco pode ser executado diversas vezes para ter maior noção dos dados

|      | Category         |
|-----:|:-----------------|
|    0 | ART_AND_DESIGN   |
| 1078 | FINANCE          |
| 2093 | FAMILY           |
| 3195 | TRAVEL_AND_LOCAL |
| 4172 | TOOLS            |
| 5105 | BUSINESS         |
| 6039 | TOOLS            |
| 6977 | SPORTS           |
| 7911 | FINANCE          |
| 8851 | LIFESTYLE        |

Podemos notar que o formato padrão para esta coluna são palavras em letras maíuscula separadas por *underline*.
Vamos filtrar os registros que não seguem esta regra:

In [138]:
# Expressão regular que checa letras maíusculas e underline
category_pattern = r"[A-Z_]"

# Cria um objeto Series com valores True|False que serve como máscara de busca 
category_mask = distinct_dataframe[CATEGORY].str.match(category_pattern)

# Usa máscara criada para filtrar as linhas com "Category" válidas
valid_category_dataframe = distinct_dataframe[category_mask]

removed_registers = (len(distinct_dataframe) - len(valid_category_dataframe))

print(f"Número de Registros Bruto: {len(distinct_dataframe):>14}")
print(f"Número de Registros Válidos: {len(valid_category_dataframe):>12}")
print(f"Número de Registros Removidos: {removed_registers:>10}")

Número de Registros Bruto:          10358
Número de Registros Válidos:        10357
Número de Registros Removidos:          1
